In [35]:
import numpy as np
import pandas as pd
import hyp3_sdk as sdk
import asf_search as asf
import pyproj
import json

from shapely.geometry import Polygon
from config import hyp3_username, hyp3_password
from med_granules import med_granules

In [3]:
# From config.py
hyp3 = sdk.HyP3(username=hyp3_username, password=hyp3_password)

# Query California Area with Polygon
Query area drawn with ASF Vertex tool, then use `asf_search` to query Sentinel-1 dataset

In [16]:
import pyproj
from shapely.geometry import Polygon

# Define the coordinates of the polygon
coords = [(-116.7506, 31.8268), (-117.8775, 33.3314), (-119.5672, 34.3467), (-120.5716, 34.366),
          (-120.9921, 35.2165), (-121.9888, 36.2279), (-122.4889, 37.1362), (-122.5961, 37.7738),
          (-122.8735, 37.9825), (-125.6648, 37.954), (-123.8274, 31.7922), (-116.7506, 31.8268)]

# Create a Polygon object
polygon = Polygon(coords)

# Use pyproj to calculate the area of the polygon
geod = pyproj.Geod(ellps='WGS84')  # Use the WGS84 ellipsoid.

area_sqm = abs(geod.geometry_area_perimeter(polygon)[0])  # area in square meters
area_sqkm = area_sqm / 1e6  # convert to square kilometers
area_sqkm

285166.7265868279

In [79]:
cali_poly = r"POLYGON((-117.3749 31.9817,-117.6997 33.3103,-119.5619 34.2627,-120.6297 34.4008,-121.2134 35.4814,-121.9635 36.1868,-122.0257 36.8362,-122.4055 37.0239,-122.8735 37.9825,-125.6648 37.954,-123.8274 31.7922,-117.3749 31.9817))"
opts = {
    'start': '2021-01-01T00:00:00Z',
    'end': '2021-12-31T00:00:00Z'
}

res = asf.geo_search(platform=[asf.PLATFORM.SENTINEL1], 
                     dataset='SENTINEL-1',
                     beamSwath='IW', 
                     intersectsWith=cali_poly, 
                     maxResults=25000, 
                     beamMode='IW',
                     polarization='VV+VH',
                     processingLevel=[' # This is also known as product_type which is confusing
                     **opts)
len(res)

12264

In [76]:
# Create a list of filenames from resulting iterable and take a sample
scenes = []
for r in res:
    geojson = r.geojson()
    sn = geojson['properties']['sceneName']
    scenes.append(sn)

scenes = np.array(list(set(scenes))) # HACK
scenes_sample = np.random.choice(scenes, size=10, replace=False)
scenes_sample

array(['S1A_IW_RAW__0SDV_20210130T015029_20210130T015101_036361_044451_865C',
       'S1A_IW_OCN__2SDV_20210123T015906_20210123T015931_036259_0440D2_1979',
       'S1B_IW_GRDH_1SDV_20210627T140720_20210627T140745_027543_0349B5_8BC1',
       'S1B_IW_RAW__0SDV_20210726T021516_20210726T021548_027959_0355FF_8A23',
       'S1B_IW_OCN__2SDV_20210203T140830_20210203T140855_025443_0307E2_C5C9',
       'S1A_IW_GRDH_1SDV_20210727T140919_20210727T140944_038964_0498F3_095D',
       'S1B_IW_GRDH_1SDV_20210325T014942_20210325T015007_026165_031F50_83B9',
       'S1B_IW_GRDH_1SDV_20210907T140839_20210907T140904_028593_036989_19CF',
       'S1A_IW_OCN__2SDV_20210403T141637_20210403T141702_037287_04648F_5B6B',
       'S1B_IW_GRDH_1SDV_20210629T014857_20210629T014922_027565_034A62_ACA8'],
      dtype='<U67')

6132

# Process existing SAR Frame with hyp3
We have captures from Stuhlmacher, Gade, et al. from Mediterranean and from Holt from California. Take reference to frames, then run through processing pipeline.

In [20]:
# From med_granules.py
med_granules = list(set(med_granules))
len(med_granules)

1735

# Processing Pipeline


In [64]:
# Create and submit RTC jobs
# EW doesn't seem to be available?

rtc_jobs = sdk.Batch()
for g in med_granules[99:]:

    try: 
        rtc_jobs += hyp3.submit_rtc_job(granule=g, 
                                        resolution=20, 
                                        scale='decibel', 
                                        speckle_filter=False,
                                        name='med_complete_iw')
    except Exception as e:
        print(e)

<Response [400]> Some requested scenes could not be found: S1A_IW_GRDH_1SDV_20160201T173732_20160201T173757_009756_00E41D_909D
<Response [400]> Some requested scenes could not be found: S1A_IW_GRDH_1SDV_20160129T061006_20160129T061031_009705_00E2AB_48EC
<Response [400]> Some requested scenes could not be found: S1A_IW_GRDH_1SDV_20160129T171307_20160129T171332_009712_00E2DB_E3EE
<Response [400]> Some requested scenes could not be found: S1A_IW_GRDH_1SDV_20160202T053657_20160202T053722_009763_00E45C_F874
<Response [400]> Some requested scenes could not be found: S1A_IW_GRDH_1SDV_20160130T175306_20160130T175331_009727_00E345_3AF9
<Response [400]> These jobs would cost 15.0 credits, but you have only 0 remaining.
<Response [400]> These jobs would cost 15.0 credits, but you have only 0 remaining.
<Response [400]> These jobs would cost 15.0 credits, but you have only 0 remaining.
<Response [400]> These jobs would cost 15.0 credits, but you have only 0 remaining.
<Response [400]> These jobs w

In [65]:
rtc_jobs = hyp3.watch(rtc_jobs)

  0%|          | 0/611 [timeout in 10800 s]


KeyboardInterrupt: 

In [81]:
rtc_jobs = hyp3.find_jobs(name='med_complete_iw')
succeeded_jobs = rtc_jobs.filter_jobs(succeeded=True, running=False, failed=False)
print(f'Number of succeeded jobs: {len(succeeded_jobs)}')
failed_jobs = rtc_jobs.filter_jobs(succeeded=False, running=False, failed=True)
print(f'Number of failed jobs: {len(failed_jobs)}')
# rtc_jobs = hyp3.watch(rtc_jobs)

Number of succeeded jobs: 664
Number of failed jobs: 1


In [79]:
hyp3.refresh(rtc_jobs)

Batch([Job.from_dict({'job_id': '71baa83d-7677-49d5-83fa-cb10b4c5d8ad', 'job_type': 'RTC_GAMMA', 'request_time': '2024-05-09T03:29:40+00:00', 'status_code': 'SUCCEEDED', 'user_id': 'kaipak', 'name': 'med_complete_iw', 'job_parameters': {'dem_matching': False, 'dem_name': 'copernicus', 'granules': ['S1A_IW_GRDH_1SDV_20150926T170448_20150926T170513_007889_00B019_39A5'], 'include_dem': False, 'include_inc_map': False, 'include_rgb': False, 'include_scattering_area': False, 'radiometry': 'gamma0', 'resolution': 20, 'scale': 'decibel', 'speckle_filter': False}, 'files': [{'filename': 'S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip', 's3': {'bucket': 'hyp3-edc-prod-contentbucket-1fv14ed36ifj6', 'key': '71baa83d-7677-49d5-83fa-cb10b4c5d8ad/S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip'}, 'size': 820778898, 'url': 'https://d3gm2hf49xd6jj.cloudfront.net/71baa83d-7677-49d5-83fa-cb10b4c5d8ad/S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip'}], 'logs': [], 'browse_images': ['https://d3g

In [ ]:
file_list = succeeded_jobs.download_files()

  0%|          | 0/664 [00:00<?, ?it/s]
S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip:   0%|          | 0/820778898 [00:00<?, ?it/s]
S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip:   1%|▏         | 10.0M/783M [00:00<01:10, 11.5MB/s]
S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip:   3%|▎         | 20.0M/783M [00:01<01:09, 11.5MB/s]
S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip:   4%|▍         | 30.0M/783M [00:02<01:09, 11.4MB/s]
S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip:   5%|▌         | 40.0M/783M [00:03<01:07, 11.5MB/s]
S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip:   6%|▋         | 50.0M/783M [00:04<01:10, 10.8MB/s]
S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip:   8%|▊         | 60.0M/783M [00:05<01:08, 11.1MB/s]
S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip:   9%|▉         | 70.0M/783M [00:06<01:07, 11.1MB/s]
S1A_IW_20150926T170448_DVP_RTC20_G_gduned_4500.zip:  10%|█         | 80.0M/783M [00:07<01:05, 11.3MB/s]
S1A_IW_20150926T170448_DVP_RTC

In [27]:
file_list

[]